In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [10]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [11]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.552892804145813
epoch:  1, loss: 0.2987568974494934
epoch:  2, loss: 0.16441497206687927
epoch:  3, loss: 0.09599409252405167
epoch:  4, loss: 0.06289730966091156
epoch:  5, loss: 0.04767049849033356
epoch:  6, loss: 0.040951646864414215
epoch:  7, loss: 0.038077838718891144
epoch:  8, loss: 0.0368734747171402
epoch:  9, loss: 0.036373261362314224
epoch:  10, loss: 0.03616387024521828
epoch:  11, loss: 0.036073021590709686
epoch:  12, loss: 0.03602994233369827
epoch:  13, loss: 0.03600570559501648
epoch:  14, loss: 0.03593648225069046
epoch:  15, loss: 0.03588350862264633
epoch:  16, loss: 0.03587307780981064
epoch:  17, loss: 0.035746656358242035
epoch:  18, loss: 0.035684891045093536
epoch:  19, loss: 0.03567744418978691
epoch:  20, loss: 0.03553331643342972
epoch:  21, loss: 0.035466086119413376
epoch:  22, loss: 0.035430099815130234
epoch:  23, loss: 0.035364601761102676
epoch:  24, loss: 0.03529410809278488
epoch:  25, loss: 0.03525877371430397
epoch:  26, loss:

In [12]:
model.eval()
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.7501114946120745
Test metrics:  R2 = 0.7733097966885041
